In [8]:
import os
import pandas as pd
import numpy as np

In [9]:
ruta_lenguaje = os.path.join("..", "Lenguaje", "inputs")
ruta_matematicas = os.path.join("..", "Matemáticas", "inputs")

# Archivos de lenguaje
archivos_lenguaje = os.listdir(ruta_lenguaje)
archivos_lenguaje = [f for f in archivos_lenguaje if "ReportePregunta" in f]
archivos_lenguaje = [os.path.join(ruta_lenguaje, f) for f in archivos_lenguaje]

# Archivos de matemáticas
archivos_matematicas = os.listdir(ruta_matematicas)
archivos_matematicas = [f for f in archivos_matematicas if "ReportePregunta" in f]
archivos_matematicas = [os.path.join(ruta_matematicas, f) for f in archivos_matematicas]

In [10]:
# Se lee primer archivo de lenguaje
df_lenguaje = pd.read_excel(archivos_lenguaje[0], header = 55)
# Se eliminan las columnas que contienen Unnamed
df_lenguaje = df_lenguaje.loc[:, ~df_lenguaje.columns.str.contains('^Unnamed')]
# Cant. se va modificando el nombre de columna por A, B, C, D, E respectivamente
df_lenguaje.columns = df_lenguaje.columns.str.replace('Cant..4', 'E', n=1)
df_lenguaje.columns = df_lenguaje.columns.str.replace('Cant..3', 'D', n=1)
df_lenguaje.columns = df_lenguaje.columns.str.replace('Cant..2', 'C', n=1)
df_lenguaje.columns = df_lenguaje.columns.str.replace('Cant..1', 'B', n=1)
df_lenguaje.columns = df_lenguaje.columns.str.replace('Cant.', 'A', n=1)
# Si tiene % se elimina columna
df_lenguaje = df_lenguaje.drop(columns=[col for col in df_lenguaje.columns if '%' in col])
# Se elimina columna P.Correcta
df_lenguaje = df_lenguaje.drop(columns=['P. Correcta'])
# Se renombra Distractor\n por Distractor
df_lenguaje = df_lenguaje.rename(columns={'Distractor\n': 'Distractor'})

# Se agrega columna de Curso, que está justo antes del .xlsx
df_lenguaje['Curso'] = '2' + archivos_lenguaje[0].split('.xlsx')[0][-1]
df_lenguaje.head(10)

,Pregunta,A,B,C,D,E,Correcta,Distractor,Curso
0,1,0,1,17,4,0,c,d,2A
1,2,19,1,1,1,0,a,"b, c, d",2A
2,3,1,6,11,3,0,b,c,2A
3,4,5,2,12,3,0,d,c,2A
4,5,5,1,10,6,0,a,c,2A
5,6,6,1,10,5,0,d,c,2A
6,7,1,13,3,5,0,b,d,2A
7,8,12,6,2,2,0,a,b,2A
8,9,2,3,13,4,0,c,d,2A
9,10,9,0,11,2,0,c,a,2A


In [11]:
# Teniendo los mismos pasos anteriores, se crea una función para transformar el archivo excel en un dataframe
def transformar_archivo_excel_a_dataframe(ruta_archivo, header):
    df = pd.read_excel(ruta_archivo, header=header)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.columns = df.columns.str.replace('Cant..4', 'E', n=1)
    df.columns = df.columns.str.replace('Cant..3', 'D', n=1)
    df.columns = df.columns.str.replace('Cant..2', 'C', n=1)
    df.columns = df.columns.str.replace('Cant..1', 'B', n=1)
    df.columns = df.columns.str.replace('Cant.', 'A', n=1)
    df = df.drop(columns=[col for col in df.columns if '%' in col])
    df = df.drop(columns=['P. Correcta'])
    df = df.rename(columns={'Distractor\n': 'Distractor'})
    df['Curso'] = '2' + ruta_archivo.split('.xlsx')[0][-1]
    return df




In [12]:
# Se lee el archivo con especificaciones de habilidad y eje temático de cada pregunta tanto en lenguaje como en matemáticas
df_especificaciones_matematicas = pd.read_excel(os.path.join("..", "Matemáticas", "inputs", "habilidades_matematicas.xlsx"))
df_especificaciones_lenguaje = pd.read_excel(os.path.join("..", "Lenguaje", "inputs", "habilidades_lenguaje.xlsx"))

# Se extraen las columnas necesarias, en este caso Habilidad y Eje temático para Matemáticas.
df_especificaciones_matematicas = df_especificaciones_matematicas[['N°', 'Habilidad', 'Eje temático']]
df_especificaciones_lenguaje = df_especificaciones_lenguaje[['N°', 'Habilidad']]

# Se crea un diccionario para mapear las habilidades y ejes temáticos en base al n° de pregunta y asignatura {"Matematicas": {n°: (Habilidad, Eje temático)}, "Lenguaje": {n°: Habilidad}}
mapa_habilidades_matematicas = {row['N°']: (row['Habilidad'], row['Eje temático']) for _, row in df_especificaciones_matematicas.iterrows()}
mapa_habilidades_lenguaje = {row['N°']: row['Habilidad'] for _, row in df_especificaciones_lenguaje.iterrows()}
mapa_habilidades = {"Matemáticas": mapa_habilidades_matematicas, "Lenguaje": mapa_habilidades_lenguaje}

In [13]:
# Se recorren los archivos de lenguaje y se van unificando en un solo dataframe
df_list = []

for archivo in archivos_lenguaje:
    df_temp = transformar_archivo_excel_a_dataframe(archivo, header=55)
    # Se agrega la columna Asignatura
    df_temp['Asignatura'] = 'LENGUAJE'
    # Se agrega la columna Habilidad en base al n° de pregunta
    df_temp['Habilidad'] = df_temp['Pregunta'].map(mapa_habilidades["Lenguaje"])
    df_list.append(df_temp)
# Se recorren  los archivos de matemáticas
for archivo in archivos_matematicas:
    df_temp = transformar_archivo_excel_a_dataframe(archivo, header=59)
    # Se agrega la columna Asignatura
    df_temp['Asignatura'] = 'MATEMATICAS'
    # Se agrega la columna Habilidad y Eje temático en base al n° de pregunta
    df_temp[['Habilidad', 'Eje temático']] = df_temp['Pregunta'].map(mapa_habilidades["Matemáticas"]).apply(pd.Series)
    df_list.append(df_temp)
# Se unen todos los dataframes en uno solo
df_final = pd.concat(df_list, ignore_index=True)
# Se agrega el calculo de porcentaje que logró responder correctamente cada pregunta
# En la columna que tiene Correcta, se busca el valor de la columna A, B, C, D o E que coincida con el valor de Correcta (en mayusculas)
df_final['Logro'] = df_final.apply(lambda x: x[x['Correcta'].upper()] / x[['A', 'B', 'C', 'D', 'E']].sum() if type(x['Correcta']) == type('str') else np.nan, axis=1)

# Se guarda el dataframe final en un archivo excel
df_final.to_excel(os.path.join("..", "Compilado", "simce_2025_preguntas.xlsx"), index=False)


Esto era para el simce_2025_estudiantes.xlsx

In [14]:
# Se toma el archivo simce_2025_estudiantes.xlsx, y se filtra sólo por las filas correspondientes a Numero_Prueba = 4, la asignatura correspondiente y el curso correspondiente.

def calcular_avance_promedio(estudiante, asignatura, numero_prueba=4):
    df_estudiante = df_final[(df_final['RUT'] == estudiante) & (df_final['Asignatura'] == asignatura) & (df_final['Numero_Prueba'] <= numero_prueba)]
    # Se guarda en dos listas el numero de prueba y el logro obtenido
    n_pruebas, logros = [], []
    for _, row in df_estudiante.iterrows():
        n_pruebas.append(row['Numero_Prueba'])
        logros.append(row['Logro'])
    # Se calcula una regresion lineal entre n_pruebas (X) y logros (Y)
    if len(n_pruebas) > 1:
        coef = np.polyfit(n_pruebas, logros, 1)
        pendiente = coef[0]
        return pendiente
    return 0

